<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Модель" data-toc-modified-id="Модель-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Модель</a></span></li><li><span><a href="#Predict" data-toc-modified-id="Predict-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Predict</a></span></li></ul></div>

# Imports

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy.stats import mode
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer

Кастомная функция для вычисления roc-auc в RandomizedSearch

In [3]:
def custom_auc(ground_truth, predictions):
    # I need only one column of predictions["0" and "1"]. You can get an error here
    # while trying to return both columns at once
    fpr, tpr, _ = roc_curve(ground_truth, predictions[:, 1], pos_label=1)
    return auc(fpr, tpr)


my_auc = make_scorer(custom_auc, greater_is_better=True, needs_proba=True)

# Обзор данных

Загружаем данные

In [7]:
file_path = '/data/share/lab04data/lab04_train.csv'
df = pd.read_csv(file_path)
print("Число строк: {}\nЧисло столбцов: {}".format(*df.shape))

Число строк: 320764
Число столбцов: 117


Посмотрим на содержимое

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320764 entries, 0 to 320763
Columns: 117 entries, Unnamed: 0 to TARGET
dtypes: float64(101), int64(3), object(13)
memory usage: 286.3+ MB


Видим, что присутствуют числовые (действительные и целые) и категорриальные признаки

Выделим имена столбцов с действительными значениями (float_cols), целыми (int_cols) и строковыми (obj_cols)

In [11]:
float_cols = df.select_dtypes(include=float64).columns.values
int_cols = df.select_dtypes(include=int64).columns.values
obj_cols = df.select_dtypes(include=object).columns.values

zero_cols = np.where((df[float_cols].fillna(0)).sum()==0)[0]
nonzero_cols = np.array([i for i in range(float_cols.shape[0]) if i not in zero_cols])
float_cols_ = float_cols[nonzero_cols]

zero_cols - действительные столбцы, у которых все значения равны 0 (индексы)

nonzero_cols - действительные столбцы, у которых есть значения не равные 0 (индексы)

float_cols_ - действительные столбцы, у которых есть значения не равные 0 (имена)

In [12]:
# Тут перечислены индексы с фичами, у которых мода равна нулю
# Почему-то на сервере scipy.stats.mode считается невыносимо долго, поэтому посчитал на своем компьютере и скопировал сюда
zero_mode = np.array([ 0,  2,  5,  7,  9, 10, 11, 13, 14, 17, 18, 19, 21, 24, 27, 29, 30,
       34, 39, 40, 42, 43, 44, 46, 47, 49, 52, 54, 60, 62, 63, 67, 68, 69,
       70, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90])
#np.where(np.array([mode(df[i]).mode[0] for i in float_cols_]) == 0)[0]
nonzero_mode = np.array([i for i in range(float_cols_.shape[0]) if i not in zero_mode])
zero_mode_cols = float_cols_[zero_mode]
nonzero_mode_cols = float_cols_[nonzero_mode]

zero_mode - действительные столбцы, у которых мода равна 0 (индексы)

nonzero_mode - действительные столбцы, у которых мода не равна 0 (индексы)

zero_mode_cols - действительные столбцы, у которых мода равна 0 (имена)

nonzero_mode_cols - действительные столбцы, у которых мода не равна 0 (имена)

Заполняем нулями пропущенные значения у столбцов, мода которых равна 0

In [14]:
df[zero_mode_cols] = df[zero_mode_cols].fillna(0)

Заполняем модами пропущенные значения у столбцов, мода которых не равна 0

In [15]:
# Почему-то на сервере scipy.stats.mode считается очень долго, поэтому тут заполнял не модами, а средними
for i in nonzero_mode_cols:
    df[i] = df[i].fillna(mean(df[i]))
# [mode(df[i]).mode[0] for i in nonzero_mode_cols]

Посмотрим теперь на данные

Столбцы с действительными значениями

In [16]:
max_rows=pd.get_option("max_rows")
pd.set_option("max_rows", 101)
max_rows

60

In [17]:
df[float_cols_].describe().T

,count,mean,std,min,25%,50%,75%,max
AMOUNT_RUB_CLO_PRC,320764.0,0.039427,0.103698,0.000000e+00,0.000000,0.000000,0.028669,1.000000e+00
APP_REGISTR_RGN_CODE,320764.0,50.940510,9.004811,0.000000e+00,50.940510,50.940510,50.940510,8.900000e+01
TURNOVER_DYNAMIC_IL_1M,320764.0,0.001312,0.029057,0.000000e+00,0.000000,0.000000,0.000000,1.000000e+00
CNT_TRAN_AUT_TENDENCY1M,320764.0,0.416497,0.147061,6.944444e-03,0.416497,0.416497,0.416497,1.000000e+00
SUM_TRAN_AUT_TENDENCY1M,320764.0,0.414412,0.157373,0.000000e+00,0.414412,0.414412,0.414412,1.000000e+00
AMOUNT_RUB_SUP_PRC,320764.0,0.076107,0.137117,0.000000e+00,0.000000,0.015898,0.095434,1.000000e+00
SUM_TRAN_AUT_TENDENCY3M,320764.0,0.688782,0.168667,4.713157e-06,0.688782,0.688782,0.688782,1.000000e+00
REST_DYNAMIC_FDEP_1M,320764.0,0.000720,0.014273,0.000000e+00,0.000000,0.000000,0.000000,1.000000e+00
CNT_TRAN_AUT_TENDENCY3M,320764.0,0.691465,0.154695,8.000000e-03,0.691465,0.691465,0.691465,1.000000e+00
REST_DYNAMIC_SAVE_3M,320764.0,0.063789,0.203693,0.000000e+00,0.000000,0.000000,0.000000,1.000000e+00


In [18]:
pd.set_option("max_rows", max_rows)

Столбцы с целочисленными значениями

In [19]:
df[int_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,320764.0,221981.072125,128175.410883,0.0,110954.75,222028.5,332823.0,443987.0
ID,320764.0,368822.072125,128175.410883,146841.0,257795.75,368869.5,479664.0,590828.0
CR_PROD_CNT_IL,320764.0,0.105551,0.430363,0.0,0.00,0.0,0.0,11.0


Столбцы со строковыми значениями

В итоге я их не использовал, поэтому никакой обработки категориальных данных не привожу

In [20]:
df[obj_cols].describe().T

,count,unique,top,freq
CLNT_TRUST_RELATION,62935,22,FRIEND,22613
APP_MARITAL_STATUS,61775,13,M,27848
APP_KIND_OF_PROP_HABITATION,53722,5,SO,25461
CLNT_JOB_POSITION_TYPE,40522,4,SPECIALIST,22741
CLNT_JOB_POSITION,190393,19053,ДИРЕКТОР,10075
APP_DRIVING_LICENSE,51896,2,N,32938
APP_EDUCATION,61665,17,H,38499
APP_TRAVEL_PASS,51896,2,N,47829
APP_CAR,51895,2,N,29760
APP_POSITION_TYPE,54780,4,SPECIALIST,33154


# Модель

Выделим датасет X со стоблбцами X_cols, на котором будем обучать модель

In [23]:
X_cols = ['CR_PROD_CNT_IL']
X_cols.extend(float_cols_[:-1])
X = df[X_cols]
print("Число строк: {}\nЧисло столбцов: {}".format(*X.shape))

Число строк: 320764
Число столбцов: 91


Столбец с таргетами

In [25]:
y = df['TARGET']
# y.shape

Разделим датасет на обучающую и отложенную выборку

In [29]:
X_train, X_test, y_train, y_test =\
                train_test_split(X, y, test_size=0.25, random_state = 0)

Настроим параметры, которые будет перебирать RandomizedSearch для RandomForest

In [58]:
# Number of trees in Random Forest
rf_n_estimators = [int(x) for x in np.linspace(10, 50, 5)]
rf_n_estimators.append(100)
# rf_n_estimators.append(2000)

# Maximum number of levels in tree
rf_max_depth = [int(x) for x in np.linspace(5, 55, 11)]
# Add the default as a possible value
rf_max_depth.append(None)

# Number of features to consider at every split
rf_max_features = ['auto', 'sqrt', 'log2']

# Criterion to split on
rf_criterion = ['mse', 'mae']

# Minimum number of samples required to split a node
rf_min_samples_split = [int(x) for x in np.linspace(2, 10, 9)]

# Minimum decrease in impurity required for split to happen
rf_min_impurity_decrease = [0.0, 0.05, 0.1]

# Method of selecting samples for training each tree
rf_bootstrap = [True, False]

# Create the grid
rf_grid = {'n_estimators': rf_n_estimators,
               'max_depth': rf_max_depth,
               'max_features': rf_max_features,
#                'criterion': rf_criterion,
               'min_samples_split': rf_min_samples_split,
               'min_impurity_decrease': rf_min_impurity_decrease,
               'bootstrap': rf_bootstrap}

Запускаем перебор гиперпараметров (удалил вывод, чтобы не засорять ноутбук)

In [ ]:
# Create the model to be tuned
rf_base = RandomForestClassifier(class_weight='balanced')

# Create the random search Random Forest
rf_random = RandomizedSearchCV(estimator = rf_base, param_distributions = rf_grid, 
                               n_iter = 200, cv = 5, verbose = 2, random_state = 0, scoring=my_auc,
                               n_jobs = 8)

# Fit the random search model
rf_random.fit(X_train, y_train)

# View the best parameters from the random search
rf_random.best_params_

Оптимальные параметры:

In [67]:
rf_random.best_params_

{'n_estimators': 100,
 'min_samples_split': 8,
 'min_impurity_decrease': 0.0,
 'max_features': 'sqrt',
 'max_depth': 55,
 'bootstrap': True}

Лучший roc-auc на валидации:

In [61]:
rf_random.best_score_

0.829233119959743

roc-auc на обучающей выборке:

In [62]:
proba_train = rf_random.best_estimator_.predict_proba(X_train)
roc_auc_score(y_train, proba_train[:,1])

0.9996240567189194

roc-auc на отложенной выборке:

In [63]:
proba_test = rf_random.best_estimator_.predict_proba(X_test)
roc_auc_score(y_test, proba_test[:,1])

0.8333447903067965

# Predict

Загрузим тестовую выборку

In [27]:
filet_path = '/data/share/lab04data/lab04_test.csv'
tdf = pd.read_csv(filet_path)
print("Число строк: {}\nЧисло столбцов: {}".format(*tdf.shape))

Число строк: 44399
Число столбцов: 116


Проделаем те же преобразования, что и при подготовке обучающей выборки

Заполняем нулями пропущенные значения у столбцов, мода которых равна 0

Заполняем модами пропущенные значения у столбцов, мода которых не равна 0

In [28]:
tdf[zero_mode_cols[:-1]] = tdf[zero_mode_cols[:-1]].fillna(0)
# Почему-то на сервере scipy.stats.mode считается очень долго, поэтому тут заполнял не модами, а средними
for i in nonzero_mode_cols:
    tdf[i] = tdf[i].fillna(mean(df[i]))
# for i in nonzero_mode_cols:
#     tdf[i] = tdf[i].fillna(mode(df[i]).mode[0]) # моды df !!!

Выделим необходимые для предикта фичи

In [29]:
X_cols = ['CR_PROD_CNT_IL']
X_cols.extend(float_cols_[:-1])
X_p = tdf[X_cols]
print("Число строк: {}\nЧисло столбцов: {}".format(*X_p.shape))

Число строк: 44399
Число столбцов: 91


Делаем прогноз на лучшем получившемся классификаторе

In [72]:
target = rf_random.best_estimator_.predict_proba(X_p)

In [76]:
res = tdf[["ID"]]

In [79]:
res["target"] = target[:,1]

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [81]:
res.columns = ["id", "target"]

In [83]:
res.head()

,id,target
0,519130,0.500440
1,234045,0.012965
2,401256,0.034848
3,551070,0.053722
4,367285,0.000000


Записываем в файл

In [85]:
res.to_csv("lab04.csv", sep='\t', index=False)

Результат: `value of your auc: 0.829503514995`

Немного не хватило для суперачивки :)

Попробуем теперь обучить модель на всем датасете и посмотрим что получится

In [31]:
clf = RandomForestClassifier(class_weight='balanced',n_estimators= 100,
 min_samples_split=8,
 min_impurity_decrease=0.0,
 max_features='sqrt',
 max_depth=55,
 bootstrap=True)

In [32]:
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=55, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=8, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

Делаем прогноз на лучшем получившемся классификаторе

In [33]:
target = clf.predict_proba(X_p)

In [34]:
res = tdf[["ID"]]

In [35]:
res["target"] = target[:,1]

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
res.columns = ["id", "target"]

In [37]:
res.head()

,id,target
0,519130,0.522827
1,234045,0.000000
2,401256,0.015559
3,551070,0.048945
4,367285,0.000000


Записываем в файл

In [38]:
res.to_csv("lab04s.csv", sep='\t', index=False)

Результат: `value of your auc: 0.832428125013`